In [2]:
import numpy as np
import pandas as pd
import json
import os

In [12]:
# places one of each 'eventType' into a json file
# this is just for testing purposes... essentially useless LOL
eventType = set()
dataDict = []

with open('Data/games/ESPORTSTMNT03_3195276.json', 'r', encoding='utf-8') as file:
    game_data = json.load(file)

# if isinstance(game_data, list):
#     game_data.reverse()

for entry in game_data:
    eventType.add(entry['eventType'])

for entry in game_data:
    if entry['eventType'] in eventType:
        eventType.remove(entry['eventType'])
        dataDict.append(entry)
    if len(eventType) == 0:
        break

with open('Data/help.json', 'w') as json_file:
    json.dump(dataDict, json_file)


FileNotFoundError: [Errno 2] No such file or directory: 'Data/games/ESPORTSTMNT03_3195276.json'

In [13]:
# modify teams.json so i get go from team id to team name

teams = dict()

with open('esports-data/teams.json', 'r', encoding='utf-8') as file:
    mapping_data = json.load(file)

for entry in mapping_data:
    teams[entry['team_id']] = entry['name']

with open('esports-data/teams_but_better.json', 'w') as json_file:
    json.dump(teams, json_file)

In [14]:
# modify mapping_data so it suites team mapping better :D
game_mapping_data = dict()

with open('esports-data/mapping_data.json', 'r', encoding='utf-8') as file:
    mapping_data = json.load(file)

for entry in mapping_data:
    game_mapping_data[entry['platformGameId']] = entry['teamMapping']

with open('esports-data/game_mapping_data.json', 'w') as json_file:
    json.dump(game_mapping_data, json_file)

In [15]:
# gets the data we want in terms of team
team_data_dict = dict()

with open('esports-data/game_mapping_data.json', 'r') as json_file:
    team_mapping_data = json.load(json_file)

for filename in os.listdir('lck_2023_summer'):
    file_path = os.path.join('lck_2023_summer', filename)

    with open(file_path, 'r', encoding='utf-8') as json_file:
        game_data = json.load(json_file)

    # we want: game_length, total_kills, total_assists, total_deaths, total_damage, total_gold, total_level, win or loss
    game_id = game_data["game_end"]['platformGameId']
    team_mapping = team_mapping_data[game_id]
    winning_team = str(game_data["game_end"]["winningTeam"])
    first_kill_team = str(game_data["champion_kill"]["killerTeamID"])
    first_turret_team = "200" if game_data["building_destroyed"]["teamID"] == 100 else "100"
    team_stats = {str(team['teamID']): team for team in game_data["end_stats"]}

    for key in team_stats:
        team_stats[key]["won"] = key == winning_team
        team_stats[key]["first_blood"] = key == first_kill_team
        team_stats[key]["first_turret"] = key == first_turret_team
        team_stats[key]["game_time"] = game_data["game_end"]["gameTime"]

    for key, value in team_stats.items():
        # print(type(key))
        team_id = team_mapping[key]
        if team_id in team_data_dict:
            team_data_dict[team_id].append(value)
        else:
            team_data_dict[team_id] = [value]

    print(team_stats)


with open('esports-data/lck_2023_team_stats.json', 'w') as json_file:
    json.dump(team_data_dict, json_file)



{'100': {'inhibKills': 1, 'towerKills': 5, 'teamID': 100, 'baronKills': 0, 'dragonKills': 3, 'assists': 32, 'totalGold': 44411, 'championsKills': 17, 'deaths': 1, 'totalDamage': 517232.474609375, 'totalLevel': 61, 'won': True, 'first_blood': True, 'first_turret': True, 'game_time': 1367407}, '200': {'inhibKills': 0, 'towerKills': 1, 'teamID': 200, 'baronKills': 0, 'dragonKills': 0, 'assists': 2, 'totalGold': 32788, 'championsKills': 1, 'deaths': 17, 'totalDamage': 377085.396484375, 'totalLevel': 56, 'won': False, 'first_blood': False, 'first_turret': False, 'game_time': 1367407}}
{'100': {'inhibKills': 2, 'towerKills': 10, 'teamID': 100, 'baronKills': 2, 'dragonKills': 2, 'assists': 40, 'championsKills': 15, 'totalGold': 69088, 'deaths': 14, 'totalDamage': 1106941.142578125, 'totalLevel': 81, 'won': True, 'first_blood': False, 'first_turret': False, 'game_time': 2163129}, '200': {'inhibKills': 0, 'towerKills': 4, 'teamID': 200, 'baronKills': 0, 'dragonKills': 3, 'assists': 31, 'champio

In [16]:
# gets the data we want in terms of team (all divided by game time)
team_data_dict = dict()

with open('esports-data/lck_2023_team_stats.json', 'r') as json_file:
    team_stats_dict = json.load(json_file)

for team_name, team_stats in team_stats_dict.items():
    for index, entry in enumerate(team_stats):
        game_time = entry["game_time"]/60000
        entry["totalGold"] = entry["totalGold"]/game_time
        entry["championsKills"] = entry["championsKills"]/game_time
        entry["assists"] = entry["assists"]/game_time
        entry["deaths"] = entry["deaths"]/game_time
        entry["totalDamage"] = entry["totalDamage"]/game_time
        entry["totalLevel"] = entry["totalLevel"]/game_time
        team_stats[index] = dict(sorted(entry.items()))

    team_data_dict[team_name] = team_stats

with open('esports-data/lck_2023_team_average_stats.json', 'w') as json_file:
    json.dump(team_data_dict, json_file)


In [34]:
# gets the data we want ignoring teams... can use something like this to train the model, based on of win or loss???
team_data_dict = list()

for filename in os.listdir('games'):
    file_path = os.path.join('games', filename)

    with open(file_path, 'r', encoding='utf-8') as json_file:
        game_data = json.load(json_file)

    winning_team = str(game_data["game_end"]["winningTeam"])
    first_kill_team = str(game_data["champion_kill"]["killerTeamID"])
    first_turret_team = "200" if game_data["building_destroyed"]["teamID"] == 100 else "100"

    for stats, mid_stats in zip(game_data["ninety_stats"], game_data["midgame_stats"]):
        team = str(stats["teamID"])
        stats = {'end_' + x : y for x, y in stats.items()}
        mid_stats = {'mid_' + x : y for x, y in mid_stats.items()}
        stats["team_id"] = stats["end_teamID"]
        stats["won"] = team == winning_team
        stats["first_blood"] = team == first_kill_team
        stats["first_turret"] = team == first_turret_team
        stats["game_time"] = game_data["game_end"]["gameTime"]
        stats["game_id"] = game_data["game_end"]["platformGameId"]

        stats.update(mid_stats)
        stats.pop("end_teamID")
        stats.pop("mid_teamID")

        team_data_dict.append(stats)


with open('esports-data/lck_2023_stats.json', 'w') as json_file:
    json.dump(team_data_dict, json_file)



In [37]:
# gets the data we want ignoring teams... can use something like this to train the model, based on of win or loss???
'''
THINGS THAT WE WANT
- gold difference
- kill difference
- death difference
- turret difference
- level difference
- dragon/baron difference
'''

team_data_dict = list()

def get_diff_stats(team1, team2, stage):
    stats = dict()
    stats["team_id"] = team1["teamID"]
    stats[stage+"golddiff"] = team1["totalGold"] - team2["totalGold"]
    stats[stage+"killdiff"] = team1["championsKills"] - team2["championsKills"]
    stats[stage+"deathdiff"] = team1["deaths"] - team2["deaths"]
    stats[stage+"turretdiff"] = team1["totalGold"] - team2["totalGold"]
    stats[stage+"leveldiff"] = team1["towerKills"] - team2["towerKills"]
    stats[stage+"dragondiff"] = team1["dragonKills"] - team2["dragonKills"]
    stats[stage+"barondiff"] = team1["baronKills"] - team2["baronKills"]

    return stats


for filename in os.listdir('games'):
    file_path = os.path.join('games', filename)

    with open(file_path, 'r', encoding='utf-8') as json_file:
        game_data = json.load(json_file)

    winning_team = str(game_data["game_end"]["winningTeam"])
    first_kill_team = str(game_data["champion_kill"]["killerTeamID"])
    first_turret_team = "200" if game_data["building_destroyed"]["teamID"] == 100 else "100"

    team1_endstats, team2_endstats = game_data["ninety_stats"][0], game_data["ninety_stats"][1]
    team1_midstats, team2_midstats = game_data["midgame_stats"][0], game_data["midgame_stats"][1]

    team1_stats = {**get_diff_stats(team1_endstats, team2_endstats, "end_"), **get_diff_stats(team1_midstats, team2_midstats, "mid_")}
    team2_stats = {**get_diff_stats(team2_endstats, team1_endstats, "end_"), **get_diff_stats(team2_midstats, team1_midstats, "mid_")}

    team1 = str(team1_stats["team_id"])
    team2 = str(team2_stats["team_id"])

    team1_stats["won"] = team1 == winning_team
    team1_stats["first_blood"] = team1 == first_kill_team
    team1_stats["first_turret"] = team1 == first_turret_team
    team1_stats["game_time"] = game_data["game_end"]["gameTime"]
    team1_stats["game_id"] = game_data["game_end"]["platformGameId"]

    team2_stats["won"] = team2 == winning_team
    team2_stats["first_blood"] = team2 == first_kill_team
    team2_stats["first_turret"] = team2 == first_turret_team
    team2_stats["game_time"] = game_data["game_end"]["gameTime"]
    team2_stats["game_id"] = game_data["game_end"]["platformGameId"]

    team_data_dict.append(team1_stats)
    team_data_dict.append(team2_stats)


with open('esports-data/lck_2023_diff_stats.json', 'w') as json_file:
    json.dump(team_data_dict, json_file)

In [38]:
team_data_dict = list()

with open('esports-data/lck_2023_stats.json', 'r') as json_file:
    stats = json.load(json_file)


stats_no_divide = {"end_inhibKills", "end_towerKills", "end_teamID", "end_baronKills", "end_dragonKills", "mid_inhibKills", "mid_towerKills", "mid_teamID", "mid_baronKills", "mid_dragonKills", "won", "first_blood", "first_turret", "game_time", "game_id"}

for entry in stats:
    game_time = entry["game_time"]/60000
    entry = {x: (y / game_time if x not in stats_no_divide else y) for x, y in entry.items()}
    entry = dict(sorted(entry.items()))

    team_data_dict.append(entry)

with open('esports-data/lck_2023_average_stats.json', 'w') as json_file:
    json.dump(team_data_dict, json_file)


In [10]:
count = 0
for filename in os.listdir('games/'):
    count += 1

print(count)

235


In [11]:
# average all the stats for each team

team_average_data = dict()

with open("Data/msi_2023_team_stats.json", 'r', encoding='utf-8') as json_file:
    team_stats = json.load(json_file)

for team, stats_list in team_stats.items():
    average_stats = {
      "inhibKills": 0,
      "towerKills": 0,
      "baronKills": 0,
      "assists": 0,
      "dragonKills": 0,
      "championsKills": 0,
      "totalGold": 0,
      "deaths": 0,
      "totalDamage": 0,
      "totalLevel": 0,
      "gameTime": 0
    }

    for key in average_stats.keys():
        average_stats[key] = sum(stats[key] for stats in stats_list)
        average_stats[key] /= len(stats_list)

    average_stats["wins"] = sum(stats["won"] for stats in stats_list)
    average_stats["losses"] = len(stats_list) - average_stats["wins"]

    team_average_data[team] = average_stats

with open('Data/msi_2023_average_team_stats.json', 'w') as json_file:
    json.dump(team_average_data, json_file)


FileNotFoundError: [Errno 2] No such file or directory: 'Data/msi_2023_team_stats.json'

In [76]:
# using this formula for all the teams
# game_grade = 0.336 — (1.437 x Deaths per min) + (0.000117 x gold per min) + (0.443 x K_A per min) + (0.264 x Level per min) + (0.000013 x TD per min)
from pprint import pprint
def calculate_grade(data):
    game_time = data["gameTime"] / 60000
    return 0.336 - (1.437 * (data["deaths"]/game_time)) + (0.000117 * (data["totalGold"]/game_time)) + (0.443 * ((data["assists"] + data["championsKills"])/game_time)) + (0.264 *  (data["totalLevel"]/game_time)) + (0.000013 * (data["totalDamage"]/game_time))

team_grade_dict = dict()

with open("Data/msi_2023_average_team_stats.json", 'r', encoding='utf-8') as json_file:
    team_average_stats = json.load(json_file)

with open("Data/teams_but_better.json", 'r', encoding='utf-8') as json_file:
    team_name_mapping = json.load(json_file)


for team, stats in team_average_stats.items():
    team_grade_dict[team_name_mapping[team]] = calculate_grade(stats)


sorted_dict = sorted(team_grade_dict.items(), key=lambda item: item[1], reverse=True)

pprint(sorted_dict)


[('Beijing JDG Intel Esports Club', 1.9653594654412818),
 ('Bilibili Gaming Pingan Bank', 1.6676915532823182),
 ('G2 Esports', 1.5754023362648737),
 ('Golden Guardians', 1.5721911369896167),
 ('Gen.G', 1.5590799162213564),
 ('T1', 1.4949498231538805),
 ('PSG Talon', 1.4322978679697056),
 ('Cloud9', 1.2741438416522382),
 ('GAM Esports', 1.2520488864903843),
 ('LOUD', 1.1627047578845437),
 ('DetonatioN FocusMe', 1.093296009574352),
 ('Movistar R7', 0.9679703133440348),
 ('MAD Lions', 0.803984159899297)]
